In [37]:
from statsmodels.robust import mad
import numpy as np
import scipy

In [38]:
a = list(range(1, 7))

In [39]:
a

[1, 2, 3, 4, 5, 6]

In [68]:
import math


class Metrics:
    def __init__(self, v):
        self.v = v
        self.size = len(v)

    def mean(self):
        return sum(self.v) / self.size

    def median(self):
        self.v.sort()
        return (
            self.v[self.size // 2]
            if len(self.v) % 2 != 0
            else Metrics(
                [self.v[(self.size // 2) - 1], self.v[(self.size // 2)]]
            ).mean()
        )

    def variance(self):
        return (
            sum(list(map(lambda x: (x - Metrics(self.v).mean()) ** 2, self.v)))
            / self.size
        )

    def stdev(self):
        return math.sqrt(Metrics(self.v).variance())

    def mean_absolute_deviation(self):
        return Metrics(
            list(map(lambda x: abs(x - Metrics(self.v).median()), self.v))
        ).median()

    def nth_percentile(self, n, interpolation='linear'):
        self.v.sort()
        
        x = (n / 100 * (self.size - 1)) + 1

        ipart = math.floor(x)
        dpart = x % 1

        return self.v[ipart - 1] + dpart * (self.v[ipart] - self.v[ipart - 1])

    def IQR(self):
        return self.nth_percentile(75) - self.nth_percentile(25)

In [69]:
f = Metrics(a)

In [70]:
f.mean()

3.5

In [71]:
f.median()

3.5

In [72]:
f.mean_absolute_deviation()

1.5

## Calculate nth percentile: 

This uses linear interpolation.

$\large v$ is a sorted sequence, 

$\large x = \frac{n}{100} * (size(v) - 1) + 1$

The equation is: 
$ \large v(x)=v_{\lfloor x\rfloor }+(x\%1)(v_{\lfloor x\rfloor +1}-v_{\lfloor x\rfloor })$

You can read more about it: [here](https://en.wikipedia.org/wiki/Percentile#Second_variant,_%7F'%22%60UNIQ--postMath-0000004A-QINU%60%22'%7F)

In [73]:
f.nth_percentile(90)

5.5

In [74]:
f.nth_percentile(99)

5.95

In [75]:
f.IQR()

2.5

## Checking with Numpy, Scipy & Statsmodels version

In [76]:
assert f.nth_percentile(90) == np.percentile(a, 90)

In [77]:
assert f.IQR() == scipy.stats.iqr(a)

In [78]:
assert f.mean_absolute_deviation() == mad(a, c=1)